### Restart and Run All Cells

In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 8
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 8, 29), datetime.date(2023, 8, 28))

In [2]:
num_business_days = BDay(2)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-08-29
yesterday: 2023-08-25


## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(48, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(190, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(48, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(48, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,AH,-12,RD15pct,SET,600,31.0,34.0,-0.75,-2.158273,0,2023-08-29
B,AWC,-33,RD25pct,SET50,4000,3.66,4.32,0.16,3.846154,0,2022-12-22
B,BBL,-31,XXX,SET50,400,155.0,170.5,1.0,0.589971,0,2023-09-06
B,BEM,-20,XXX,SET50,2500,7.8,8.8,0.1,1.149425,0,2022-12-22
B,BH,-8,XXX,SET50,100,250.0,258.0,3.0,1.176471,1,2022-12-22
B,CK,-11,XXX,SET100,1000,22.1,23.2,0.2,0.869565,0,2022-12-22
B,CPN,-18,XXX,SET50,300,64.5,69.0,-0.5,-0.719424,0,2022-12-22
B,DELTA,-58,XXX,SET50,200,90.0,109.0,2.5,2.347418,0,2022-12-22
B,DIF,-14,RD40pct,SET,3000,8.65,9.35,0.0,0.0,0,2023-08-15
B,HANA,-15,XXX,SET100,400,59.0,62.75,1.0,1.619433,0,2022-12-22
B,ICHI,-20,XXX,SET,1500,14.3,16.3,-0.4,-2.39521,0,2022-12-22
B,III,-12,XXX,SET,2000,10.8,12.0,0.2,1.694915,0,2022-12-22
B,JASIF,-7,RD35pct,SET,4000,6.5,6.85,-0.05,-0.724638,1,2023-08-21
B,JMART,-31,RD45pct,SET50,3000,18.9,22.0,-0.4,-1.785714,0,2023-08-24
B,JMT,-26,RD30pct,SET50,1800,39.75,46.25,-0.5,-1.06

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
34,S,AIMIRT,1,DOS,SET,10000,11.10,11.00,0.10,0.92,1,2023-08-23,111000.00
47,S,WHART,1,COS,SET,10000,10.60,10.50,0.00,0.00,2,2023-08-21,106000.00


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
47,S,WHART,1,COS,SET,10000,10.60,10.50,0.00,0.00,2,2023-08-21,106000.00


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
4,B,BH,-8,XXX,SET50,100,250.00,258.00,3.00,1.18,1,2022-12-22,25000.00
12,B,JASIF,-7,RD35pct,SET,4000,6.50,6.85,-0.05,-0.72,1,2023-08-21,26000.00
15,B,KTB,-6,XXX,SET50,1000,18.80,19.40,0.00,0.00,1,2023-04-20,18800.00
17,B,MCS,-9,RD60pct,SET,4000,6.00,6.45,0.10,1.57,1,2023-03-17,24000.00
20,B,PSH,-12,XXX,SET,2000,12.20,13.40,0.20,1.52,1,2023-03-10,24400.00
23,B,QH,-12,XXX,SET100,10000,2.10,2.34,0.04,1.74,1,2023-08-23,21000.00
28,B,SIRI,-5,XXX,SET,10000,1.90,1.95,-0.03,-1.52,1,2022-12-22,19000.00
34,S,AIMIRT,1,DOS,SET,10000,11.10,11.00,0.10,0.92,1,2023-08-23,111000.00
39,S,GVREIT,6,DOS,SET,6000,7.40,7.10,-0.10,-1.39,1,2023-08-28,44400.00
45,S,TFFIF,5,XXX,SET,20000,7.75,7.50,0.00,0.00,1,2023-09-01,155000.00


In [17]:
pd.options.display.max_rows = 35
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,AH,-12,RD15pct,SET,600,31.00,34.00,-0.75,-2.16,0,2023-08-29,18600.00
1,B,AWC,-33,RD25pct,SET50,4000,3.66,4.32,0.16,3.85,0,2022-12-22,14640.00
2,B,BBL,-31,XXX,SET50,400,155.00,170.50,1.00,0.59,0,2023-09-06,62000.00
3,B,BEM,-20,XXX,SET50,2500,7.80,8.80,0.10,1.15,0,2022-12-22,19500.00
5,B,CK,-11,XXX,SET100,1000,22.10,23.20,0.20,0.87,0,2022-12-22,22100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,S,KCE,47,COS,SET100,1000,61.75,50.00,1.00,2.04,0,2023-08-21,61750.00
42,S,PTG,12,5pct,SET100,3600,12.10,10.90,0.00,0.00,0,2022-12-22,43560.00
43,S,RCL,55,COS,SET100,3000,30.50,21.70,-0.20,-0.91,0,2023-08-24,91500.00
44,S,SCC,68,COS,SET50,300,385.00,317.00,2.00,0.63,0,2023-08-09,115500.00


In [18]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
7,B,DELTA,-58,XXX,SET50,200,90.00,109.00,2.50,2.35,0,2022-12-22,18000.00
21,B,PTTEP,-49,XXX,SET50,200,134.50,159.00,-1.00,-0.62,0,2023-08-15,26900.00


In [19]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [20]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [21]:
df['percent'] = df['%change']

In [22]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
0,B,AH,-12,RD15pct,SET,600,31.00,34.00,-0.75,-2.16,0,2023-08-29,18600.00,-2.16
10,B,ICHI,-20,XXX,SET,1500,14.30,16.30,-0.40,-2.40,0,2022-12-22,21450.00,-2.40
22,B,PTTGC,-10,RD4,SET50,600,34.50,37.00,-1.00,-2.63,0,2023-08-29,20700.00,-2.63
40,S,IVL,-614,RD30pct,SET50,800,0.00,28.50,-0.75,-2.56,0,2023-08-28,0.00,-2.56


In [23]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
1,B,AWC,-33,RD25pct,SET50,4000,3.66,4.32,0.16,3.85,0,2022-12-22,14640.00,3.85
7,B,DELTA,-58,XXX,SET50,200,90.00,109.00,2.50,2.35,0,2022-12-22,18000.00,2.35
18,B,NER,-17,RD40pct,SET,3000,4.44,4.78,0.10,2.14,0,2023-08-25,13320.00,2.14
41,S,KCE,47,COS,SET100,1000,61.75,50.00,1.00,2.04,0,2023-08-21,61750.00,2.04


In [24]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
28,B,SIRI,-5,XXX,SET,10000,1.90,1.95,-0.03,-1.52,1,2022-12-22,19000.00,-1.52
15,B,KTB,-6,XXX,SET50,1000,18.80,19.40,0.00,0.00,1,2023-04-20,18800.00,0.00
12,B,JASIF,-7,RD35pct,SET,4000,6.50,6.85,-0.05,-0.72,1,2023-08-21,26000.00,-0.72
4,B,BH,-8,XXX,SET50,100,250.00,258.00,3.00,1.18,1,2022-12-22,25000.00,1.18
17,B,MCS,-9,RD60pct,SET,4000,6.00,6.45,0.10,1.57,1,2023-03-17,24000.00,1.57
22,B,PTTGC,-10,RD4,SET50,600,34.50,37.00,-1.00,-2.63,0,2023-08-29,20700.00,-2.63
16,B,M,-10,XXX,SET,500,46.00,48.50,0.50,1.04,0,2022-12-22,23000.00,1.04
5,B,CK,-11,XXX,SET100,1000,22.10,23.20,0.20,0.87,0,2022-12-22,22100.00,0.87
25,B,SC,-12,XXX,SET,5000,4.10,4.34,0.02,0.46,0,2023-08-23,20500.00,0.46
23,B,QH,-12,XXX,SET100,10000,2.10,2.34,0.04,1.74,1,2023-08-23,21000.00,1.74


### Sell recently purchased stocks if profitable

In [25]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(108, 4)

In [26]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [27]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(105, 8)

In [28]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit
1,AIMIRT,2023-08-17,10000,11.00,11.00,0.00,0.92,0.00
95,TFFIF,2023-07-25,10000,7.45,7.50,500.00,0.00,0.67
107,WHART,2022-05-17,10000,10.50,10.50,0.00,0.00,0.00


### Profit per Day

In [29]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 8
ORDER BY sells.date DESC, name


In [30]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,JMART,2023,8,20.60,19.60,1.00,"1,200","24,720.00","23,520.00","1,200.00",5.10,126
1,ICHI,2023,8,15.90,14.30,1.60,"1,400","22,260.00","20,020.00","2,240.00",11.19,59
2,TTB,2023,8,1.76,1.64,0.12,"15,000","26,400.00","24,600.00","1,800.00",7.32,26
3,JMART,2023,8,18.30,17.00,1.30,"3,600","65,880.00","61,200.00","4,680.00",7.65,41
4,JMT,2023,8,43.50,39.75,3.75,"1,800","78,300.00","71,550.00","6,750.00",9.43,57


In [31]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [32]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,JMART,2023,8,20.60,19.60,1.00,"1,200","24,720.00","23,520.00","1,200.00",5.10,126,0.040492
1,ICHI,2023,8,15.90,14.30,1.60,"1,400","22,260.00","20,020.00","2,240.00",11.19,59,0.189641
2,TTB,2023,8,1.76,1.64,0.12,"15,000","26,400.00","24,600.00","1,800.00",7.32,26,0.281426
3,JMART,2023,8,18.30,17.00,1.30,"3,600","65,880.00","61,200.00","4,680.00",7.65,41,0.186514
4,JMT,2023,8,43.50,39.75,3.75,"1,800","78,300.00","71,550.00","6,750.00",9.43,57,0.165508


In [33]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 0.17%
